<a href="https://colab.research.google.com/github/pktensor/image-predictor/blob/main/artifact_replacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install opencv-python torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [18]:
import cv2
import torch
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load cricket ball image and create a mask
cricket_ball = cv2.imread('/content/Pink-Cricket-Ball.jpg', cv2.IMREAD_UNCHANGED)

# Check if the image has an alpha channel
if cricket_ball.shape[2] == 4:  # Image has an alpha channel (e.g., PNG)
    cricket_ball_alpha = cricket_ball[:, :, 3]
    cricket_ball = cricket_ball[:, :, :3]  # Extract RGB channels
    cricket_ball_mask = cv2.threshold(cricket_ball_alpha, 1, 255, cv2.THRESH_BINARY)[1]
else:  # Image does not have an alpha channel (e.g., JPG)
    # Create a mask assuming the ball is a circle or using other methods
    cricket_ball_gray = cv2.cvtColor(cricket_ball, cv2.COLOR_BGR2GRAY)
    _, cricket_ball_mask = cv2.threshold(cricket_ball_gray, 1, 255, cv2.THRESH_BINARY_INV)

# cricket_ball_alpha = cricket_ball[:, :, 3]
# cricket_ball = cricket_ball[:, :, :3]
# cricket_ball_mask = cv2.threshold(cricket_ball_alpha, 1, 255, cv2.THRESH_BINARY)[1]

# Load video
video_path = "/content/Football-Shorts.mp4"
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Function to replace the football with the cricket ball
def replace_football_with_cricketball(frame, cricket_ball, cricket_ball_mask, position):
    x, y, w, h = position
    cricket_ball_resized = cv2.resize(cricket_ball, (w, h))
    cricket_ball_mask_resized = cv2.resize(cricket_ball_mask, (w, h))

    roi = frame[y:y+h, x:x+w]

    roi[np.where(cricket_ball_mask_resized)] = 0
    roi += cricket_ball_resized

    frame[y:y+h, x:x+w] = roi
    return frame

# Main processing loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()

    for *box, conf, cls in detections:
        if cls == 32:  # Class ID for sports ball
            x1, y1, x2, y2 = map(int, box)
            position = (x1, y1, x2 - x1, y2 - y1)
            frame = replace_football_with_cricketball(frame, cricket_ball, cricket_ball_mask, position)

    # Write frame to output video
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-10 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning:

In [1]:
!pip install opencv-python mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 12.8 MB/s eta 0:00:00


In [8]:
import cv2
import numpy as np
import mediapipe as mp

# Load video
video_path = "/content/Y21WdVpHVnlYMkZ3YVY5allXTm9aVjlZTFhSUGJFaEtjR2xOWTE4eE1EZ3djRFl3WDJabWVYUS9YLXRPbEhKcGlNY19qdWJlci1ZbWlKV3lFT2M_out.mp4"
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('new_output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_objectron = mp.solutions.objectron

# Object detection and replacement
objectron = mp_objectron.Objectron(static_image_mode=False, max_num_objects=4, model_name='Cup')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = objectron.process(frame_rgb)

    if results.detected_objects:
        for detected_object in results.detected_objects:
            bbox = detected_object.location_data.relative_bounding_box
            h, w, _ = frame.shape
            xmin = int(bbox.xmin * w)
            ymin = int(bbox.ymin * h)
            xmax = int(bbox.width * w + xmin)
            ymax = int(bbox.height * h + ymin)

            # Load new object image
            new_object = cv2.imread('/content/Mens-Magna-Pink-Cricket-Ball.jpg', cv2.IMREAD_UNCHANGED)
            new_object = cv2.resize(new_object, (xmax-xmin, ymax-ymin))

            # Blend the new object onto the frame
            alpha_s = new_object[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s
            for c in range(0, 3):
                frame[ymin:ymax, xmin:xmax, c] = (alpha_s * new_object[:, :, c] + alpha_l * frame[ymin:ymax, xmin:xmax, c])

    # Write frame to output video
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

In [12]:
import cv2
import numpy as np

# Load video
video_path = "/content/Y21WdVpHVnlYMkZ3YVY5allXTm9aVjlZTFhSUGJFaEtjR2xOWTE4eE1EZ3djRFl3WDJabWVYUS9YLXRPbEhKcGlNY19qdWJlci1ZbWlKV3lFT2M_out.mp4"
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('new_short_output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Load cricket ball image and create a mask
cricket_ball = cv2.imread('/content/Mens-Magna-Pink-Cricket-Ball.jpg', cv2.IMREAD_UNCHANGED)

# Check if the image has an alpha channel
if cricket_ball.shape[2] == 4:
    cricket_ball_alpha = cricket_ball[:, :, 3]
    cricket_ball_mask = cv2.threshold(cricket_ball_alpha, 1, 255, cv2.THRESH_BINARY)[1]
else:
    # If no alpha channel, create a mask assuming the ball is a circle
    cricket_ball_gray = cv2.cvtColor(cricket_ball, cv2.COLOR_BGR2GRAY)
    _, cricket_ball_mask = cv2.threshold(cricket_ball_gray, 1, 255, cv2.THRESH_BINARY_INV)
    # This might need adjustment depending on the ball image


cricket_ball = cricket_ball[:, :, :3]
cricket_ball = cv2.resize(cricket_ball, (50, 50))  # Resize cricket ball as needed
# cricket_ball_mask = cv2.threshold(cricket_ball_alpha, 1, 255, cv2.THRESH_BINARY)[1]

# Function to replace the football with the cricket ball
def replace_football_with_cricketball(frame, cricket_ball, cricket_ball_mask, position):
    x, y = position
    y1, y2 = y, y + cricket_ball.shape[0]
    x1, x2 = x, x + cricket_ball.shape[1]

    # Ensure the coordinates are within frame boundaries
    if y2 > frame.shape[0] or x2 > frame.shape[1]:
        return frame

    roi = frame[y1:y2, x1:x2]

    # Resize the mask to match the cricket ball's dimensions
    resized_mask = cv2.resize(cricket_ball_mask.astype(np.uint8), (cricket_ball.shape[1], cricket_ball.shape[0]))

    roi[np.where(resized_mask)] = 0
    roi += cricket_ball

    frame[y1:y2, x1:x2] = roi
    return frame

# Main processing loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Assuming the football is at a fixed position for simplicity
    # In a more complex scenario, you can use object tracking to find the football
    football_position = (100, 100)  # Example position

    frame = replace_football_with_cricketball(frame, cricket_ball, cricket_ball_mask, football_position)

    # Write frame to output video
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
